### Download Video

In [ ]:
# 1. Video herunterladen (optional - bei Verwendung von YouTube)
video_url = "https://www.youtube.com/watch?v=EkrxauaWNGc"
yt = YouTube(video_url)
yt.streams.filter(progressive=True, file_extension='mp4').first().download(output_path=".", filename="input_video.mp4")

# 2. Gesichtserkennung im Video
video_path = 'input_video.mp4'
target_image = 'target_face.jpg'  # Bild der Zielperson
target_encoding = face_recognition.face_encodings(face_recognition.load_image_file(target_image))[0]

video = cv2.VideoCapture(video_path)
frame_rate = int(video.get(cv2.CAP_PROP_FPS))
matching_frames = []

# frame_number = 0
# while video.isOpened():
#     ret, frame = video.read()
#     if not ret:
#         break
#     frame_number += 1
#     if frame_number % frame_rate == 0:  # Jeden Sekunde-Frame überprüfen
#         rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         face_locations = face_recognition.face_locations(rgb_frame)
#         face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
#         for face_encoding in face_encodings:
#             if face_recognition.compare_faces([target_encoding], face_encoding)[0]:
#                 matching_frames.append(frame_number / frame_rate)
# video.release()

# # 3. Audio-Extraktion
# ffmpeg.input(video_path).output('audio.wav').run()

# # 4. Stimmerkennung und Transkription mit Whisper
# from transformers import WhisperProcessor, WhisperForConditionalGeneration

# # Whisper model laden
# processor = WhisperProcessor.from_pretrained("openai/whisper-large")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

# # Audio transkribieren
# def transcribe_audio(audio_path):
#     audio_input = processor(audio_path, return_tensors="pt")
#     predicted_ids = model.generate(audio_input["input_values"])
#     transcript = processor.decode(predicted_ids[0])
#     return transcript

# transcript = transcribe_audio("audio.wav")

# # 5. Emotionale Analyse der Audioaufnahme (wav2vec2)
# from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification

# # Wav2Vec2 model laden für emotionale Analyse
# wav2vec2_processor = Wav2Vec2Processor.from_pretrained("ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition")
# wav2vec2_model = Wav2Vec2ForSequenceClassification.from_pretrained("ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition")

# def analyze_audio_emotion(audio_path):
#     audio_input = wav2vec2_processor(audio_path, return_tensors="pt", sampling_rate=16000)
#     audio_emotion = wav2vec2_model(audio_input["input_values"]).logits.argmax(axis=-1).item()
#     emotions = ["angry", "happy", "sad", "neutral"]  # Emotionen-Kategorien
#     return emotions[audio_emotion]

# audio_emotion = analyze_audio_emotion("audio.wav")

# # 6. Emotionserkennung aus Bild (Gesicht)
# emotion_detector = FER(mtcnn=True)
# frame_emotions = [emotion_detector.detect_emotions(frame) for frame in matching_frames]

# # 7. Emotionserkennung aus Text (Sentiment-Analyse)
# sentiment_pipeline = pipeline("sentiment-analysis")
# text_emotions = sentiment_pipeline(transcript)

# # 8. Ergebnisse speichern
# results = {
#     "scenes": [
#         {"time": time, "frame_emotion": emotion, "text_emotion": text_emotions, "audio_emotion": audio_emotion}
#         for time, emotion in zip(matching_frames, frame_emotions)
#     ]
# }

# import json
# with open('output.json', 'w') as f:
#     json.dump(results, f)

# print("Emotionserkennung abgeschlossen. Ergebnisse gespeichert in 'output.json'.")


### Frames where Target Face is visible

In [ ]:
## Extracting Frames of Target Face

### Frames where Target Voice is audible

In [ ]:
## Exracting Frames of Target Voice

# Extracting Features

### Audio

In [13]:
from transformers import AutoModelForAudioClassification, AutoProcessor, pipeline
import torch
import librosa

# pipe = pipeline("audio-classification", model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition")     
pipe = pipeline("audio-classification", model="Hatman/audio-emotion-detection")     
# Load your audio file using librosa
audio_file = "audio.wav"
audio, rate = librosa.load(audio_file, sr=16000)  # Ensure it's resampled to 16kHz

# Get the emotion prediction
result = pipe(audio)

# Print the result
print(result)


config.json:   0%|          | 0.00/2.65k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

Could not load the `decoder` for Hatman/audio-emotion-detection. Defaulting to raw CTC. Error: No module named 'kenlm'
Try to install `kenlm`: `pip install kenlm
Try to install `pyctcdecode`: `pip install pyctcdecode
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
  audio, rate = librosa.load(audio_file, sr=16000)  # Ensure it's resampled to 16kHz

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[{'score': 0.5844987034797668, 'label': 'Happy'}, {'score': 0.19212304055690765, 'label': 'Angry'}, {'score': 0.17521733045578003, 'label': 'Disgusted'}, {'score': 0.03364561125636101, 'label': 'Fearful'}, {'score': 0.0062223924323916435, 'label': 'Suprised'}]


### Text

In [15]:
# Whisper for text transcription
# Detecting Feeling in Text

import whisper
from transformers import pipeline

# Schritt 1: Audio transkribieren
def transcribe_audio(audio_path):
    # Whisper-Modell laden
    model = whisper.load_model("base")  # Du kannst auch "small", "medium", "large" wählen
    result = model.transcribe(audio_path)
    return result["text"]

# Schritt 2: Emotionen im Text analysieren
def analyze_emotion(text):
    # Emotionserkennungsmodell laden
    emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
    emotions = emotion_pipeline(text)
    return emotions

# Audio-Datei Pfad
audio_file = "audio.wav"

# Transkription
print("Transkribiere die Audiodatei...")
transcribed_text = transcribe_audio(audio_file)
print(f"Transkribierter Text: {transcribed_text}")

# Emotionserkennung
print("Analysiere die Emotionen im Text...")
emotions = analyze_emotion(transcribed_text)
print(emotions)
print("Erkannte Emotionen mit Scores:")
for emotion in emotions[0]:  # emotions[0] enthält die Liste der Emotionen
    print(f"{emotion['label']}: {emotion['score']:.2f}")


Transkribiere die Audiodatei...


  checkpoint = torch.load(fp, map_location=device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Transkribierter Text:  Okay, sir, you agree with me. You don't actually have to go to Bronze Circle to get Google Maps to generate these images. All you have to do is go to the website.
Analysiere die Emotionen im Text...


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
  warnings.warn(



[[{'label': 'anger', 'score': 0.010466489009559155}, {'label': 'disgust', 'score': 0.010573158971965313}, {'label': 'fear', 'score': 0.0021569335367530584}, {'label': 'joy', 'score': 0.0030502486042678356}, {'label': 'neutral', 'score': 0.9656197428703308}, {'label': 'sadness', 'score': 0.0030216046143323183}, {'label': 'surprise', 'score': 0.0051116799004375935}]]
Erkannte Emotionen mit Scores:
anger: 0.01
disgust: 0.01
fear: 0.00
joy: 0.00
neutral: 0.97
sadness: 0.00
surprise: 0.01


### Video

In [ ]:
# Image/Video
import cv2
from deepface import DeepFace


def extract_frames(video_path, frame_rate=1):
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    interval = int(fps / frame_rate)  # Frame interval
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % interval == 0:
            frames.append(frame)
        frame_count += 1
    cap.release()
    return frames


def analyze_emotions(frames):
    emotions = []
    for idx, frame in enumerate(frames):
        result = DeepFace.analyze(frame, actions=["emotion"], enforce_detection=False)
        emotions.append((idx, result['dominant_emotion']))
    return emotions

video_file = "video.mp4"

# Extrahiere Frames
print("Extrahiere Frames aus dem Video...")
frames = extract_frames(video_file, frame_rate=1)  # 1 Frame pro Sekunde

# Analysiere Emotionen
print("Analysiere Emotionen in den Frames...")
emotions = analyze_emotions(frames)

# Ergebnisse ausgeben
print("Ergebnisse der Emotionserkennung:")
for idx, emotion in emotions:
    print(f"Frame {idx}: {emotion}")


In [11]:
import cv2
from fer import FER
from collections import Counter

def detect_emotions_in_video(video_path):
    """
    Detect and count emotions in a video
    
    Args:
        video_path (str): Path to the input video file
    """
    # Initialize the FER detector
    emotion_detector = FER(mtcnn=True)
    
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not video_capture.isOpened():
        print("Error: Could not open video file")
        return None
    
    # Prepare to store emotion results
    emotion_counts = Counter()
    total_detected_frames = 0
    
    # Frame-by-frame emotion detection
    while True:
        # Read a frame from the video
        ret, frame = video_capture.read()
        
        # Break the loop if no more frames
        if not ret:
            break
        
        # Detect emotions in the frame
        detected_emotions = emotion_detector.detect_emotions(frame)
        
        # Process detected emotions
        if detected_emotions:
            total_detected_frames += 1
            for emotion_result in detected_emotions:
                # Get dominant emotion
                emotions = emotion_result['emotions']
                dominant_emotion = max(emotions, key=emotions.get)
                emotion_counts[dominant_emotion] += 1
    
    # Release resources
    video_capture.release()
    
    # Print emotion occurrence summary
    print("\nEmotion Occurrence Summary:")
    for emotion, count in emotion_counts.items():
        percentage = (count / total_detected_frames) * 100 if total_detected_frames > 0 else 0
        print(f"{emotion}: {count} times ({percentage:.2f}%)")
    
    return dict(emotion_counts)

In [12]:
video_path = "input_video.mp4"
detect_emotions_in_video(video_path)

  state_dict = torch.load(state_dict_path)

  state_dict = torch.load(state_dict_path)

  state_dict = torch.load(state_dict_path)




Emotion Occurrence Summary:
neutral: 414 times (128.17%)
happy: 415 times (128.48%)
angry: 323 times (100.00%)
sad: 296 times (91.64%)
fear: 85 times (26.32%)
surprise: 2 times (0.62%)


{'neutral': 414,
 'happy': 415,
 'angry': 323,
 'sad': 296,
 'fear': 85,
 'surprise': 2}